# MultiHandler: Synchronized Multi-Task Acquisition

The `MultiHandler` class synchronizes multiple NI-DAQmx tasks for complex acquisition scenarios.

**Key capabilities:**

- Synchronize multiple input/output tasks via hardware triggers (FINITE mode)
- Software-triggered acquisition via pyTrigger (CONTINUOUS mode, single task)
- Comprehensive validation pipeline ensures task compatibility
- Thread-safe operations (RLock)

**Architecture note:** `MultiHandler` is NOT a subclass of `DAQHandler`. It operates on pre-configured nidaqmx tasks.

---

## Setup: Hardware Detection

This cell tries to import nidaqmx. If unavailable, it sets up a lightweight mock.

In [ ]:
# Attempt to import nidaqmx; fall back to mock if unavailable
try:
    import nidaqmx
    from nidaqmx import constants
    HW_AVAILABLE = True
    print("\u2713 Real NI-DAQmx hardware available")
except ImportError:
    print("\u2717 NI-DAQmx not found; using mock mode")
    HW_AVAILABLE = False
    
    # Minimal mock for MultiHandler demonstration
    from unittest.mock import MagicMock
    import sys
    
    # Mock nidaqmx module and constants
    nidaqmx = MagicMock()
    constants = MagicMock()
    
    # Mock system device discovery
    mock_device = MagicMock()
    mock_device.name = "Dev1"
    mock_device.product_type = "PCIe-6320"
    nidaqmx.system.System.local.return_value.devices = [mock_device]
    nidaqmx.system.System.local.return_value.tasks.task_names = []
    
    # Mock task behavior
    mock_task = MagicMock()
    mock_task.name = "mock_task"
    mock_task.channel_names = ["Dev1/ai0", "Dev1/ai1"]
    mock_task.is_task_done.return_value = False
    mock_task.timing.samp_clk_rate = 25600.0
    mock_task.timing.samp_quant_samp_mode.name = "CONTINUOUS"
    mock_task.triggers.start_trigger.trig_type.name = "NONE"
    mock_task.devices = [mock_device]
    
    nidaqmx.task.Task.return_value = mock_task
    nidaqmx.constants = constants
    
    # Mock get_connected_devices and get_task_by_name
    sys.modules['nidaqmx'] = nidaqmx
    
    # Create mock utils functions
    def mock_get_connected_devices():
        return {"Dev1"}
    
    def mock_get_task_by_name(name):
        return mock_task

import numpy as np

## Basic Construction

`MultiHandler` has no constructor arguments. All configuration is done via `configure()`.

In [ ]:
from nidaqwrapper import MultiHandler

# Create an instance
adv = MultiHandler()

print(f"Configured: {adv._configured}")
print(f"Connected: {adv._connected}")
print(f"Trigger type: {adv.trigger_type}")

## Task Configuration

`configure()` accepts mixed task types:

- `nidaqmx.task.Task` — raw nidaqmx task object
- `AITask` / `AOTask` — wrapper task classes
- `str` — task name from NI MAX

All tasks are resolved to `nidaqmx.task.Task` objects and validated.

In [ ]:
from nidaqwrapper import AITask, AOTask

# Example: create two AITask objects
task1 = AITask("vibration_test", sample_rate=25600)
task1.add_channel(
    channel_name="accel_x",
    device_ind=0,
    channel_ind=0,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g"
)

task2 = AITask("pressure_test", sample_rate=25600)
task2.add_channel(
    channel_name="pressure_1",
    device_ind=0,
    channel_ind=1,
    units="V"
)

# Configure MultiHandler with these tasks
success = adv.configure(input_tasks=[task1, task2])
print(f"Configuration success: {success}")
print(f"Input tasks: {len(adv.input_tasks)}")
print(f"Output tasks: {len(adv.output_tasks)}")

## Validation Pipeline

When you call `configure()`, MultiHandler runs a sequential validation pipeline:

1. **Type validation** — input_tasks and output_tasks must be lists containing only valid task types
2. **Task resolution** — strings become nidaqmx tasks (via NI MAX), AITask/AOTask objects call `.start(start_task=False)` and extract `.task`
3. **Validity check** — all tasks must be open and have at least one channel
4. **Sample rate consistency** — all tasks within a group must share the same rate
5. **Timing configuration consistency** — clock source, clock rate, and samples_per_channel must match
6. **Trigger consistency** — all tasks must have the same trigger type (hardware or none)
7. **Acquisition mode compatibility** — FINITE requires hardware trigger; CONTINUOUS requires software trigger

If any validation step fails, `configure()` returns `False` and the instance state is not updated.

## Connection and Device Discovery

`connect()` calls `ping()` to verify all required devices are present.

In [ ]:
# Connect to hardware
connected = adv.connect()
print(f"Connected: {connected}")
print(f"Required devices: {adv.required_devices}")

# Ping can be called separately
available = adv.ping()
print(f"All devices available: {available}")

## Hardware-Triggered Acquisition

FINITE acquisition mode with hardware triggers:

1. All tasks start (arm and wait for trigger)
2. Hardware trigger fires
3. `acquire()` dispatches to `acquire_with_hardware_trigger()`
4. Each task reads `READ_ALL_AVAILABLE` samples
5. All tasks stop

**Returns:** Nested dict `{task_name: {channel_name: np.ndarray}}`

In [ ]:
# Example hardware-triggered acquisition (pseudo-code)
# Assumes tasks were configured with FINITE mode + hardware trigger

# data = adv.acquire()
# for task_name, channels in data.items():
#     print(f"Task: {task_name}")
#     for channel_name, samples in channels.items():
#         print(f"  {channel_name}: {samples.shape}")

print("Hardware-triggered acquisition requires FINITE mode + hardware trigger.")
print("Example output structure:")
print("{")
print("  'vibration_test': {")
print("    'accel_x': array([...]),  # shape: (n_samples,)")
print("  },")
print("  'pressure_test': {")
print("    'pressure_1': array([...]),")
print("  }")
print("}")

## Software-Triggered Acquisition

CONTINUOUS acquisition mode with pyTrigger (single input task only):

1. Call `set_trigger(n_samples, trigger_channel, trigger_level, trigger_type='abs', presamples=0)`
2. `acquire()` dispatches to `acquire_with_software_trigger()`
3. Task starts, data is read in a loop and fed to pyTrigger ring buffer
4. When trigger condition is met, pyTrigger signals completion
5. Task stops

**Returns:** Dict with channel names + `'time'` key (when `return_dict=True`)

In [ ]:
# Example software-triggered acquisition (pseudo-code)
# Assumes single input task with CONTINUOUS mode

# adv.set_trigger(
#     n_samples=25600,
#     trigger_channel=0,
#     trigger_level=0.5,
#     trigger_type='abs',
#     presamples=0
# )
# data = adv.acquire()
# print(data.keys())  # ['accel_x', 'time']

print("Software-triggered acquisition requires:")
print("  - CONTINUOUS mode")
print("  - Single input task only")
print("  - pyTrigger installed")
print("")
print("Example output structure:")
print("{")
print("  'accel_x': array([...]),  # shape: (n_samples,)")
print("  'time': array([...]),     # shape: (n_samples,)")
print("}")

## Trigger Configuration

`set_trigger()` configures the pyTrigger for software-triggered acquisition.

**Parameters:**

- `n_samples` (int) — number of samples to acquire after trigger event
- `trigger_channel` (int) — index of channel to monitor
- `trigger_level` (float) — threshold that activates trigger
- `trigger_type` (str) — detection mode (default: `'abs'`)
- `presamples` (int) — samples to retain before trigger event (default: 0)

**Notes:**

- Must be called before `acquire()` in software trigger mode
- Only valid when `trigger_type == 'software'`
- Requires at least one input task to be configured

## Lifecycle Management

MultiHandler provides standard lifecycle methods:

- `connect()` → bool — verify devices are present
- `disconnect()` → bool — close all tasks, release resources
- `ping()` → bool — check device availability

All methods are thread-safe (RLock).

In [ ]:
# Disconnect when done
disconnected = adv.disconnect()
print(f"Disconnected: {disconnected}")
print(f"Connected state: {adv._connected}")

## Complete Example: Hardware-Triggered Multi-Task

This example demonstrates synchronized acquisition of two input tasks with a hardware trigger.

In [ ]:
# Pseudo-code for hardware-triggered multi-task acquisition

# 1. Create and configure tasks with FINITE mode + hardware trigger
# task1 = AITask("accel", sample_rate=25600)
# task1.add_channel("x", device_ind=0, channel_ind=0, ...)
# task1.task.timing.cfg_samp_clk_timing(
#     rate=25600,
#     sample_mode=constants.AcquisitionType.FINITE,
#     samps_per_chan=25600
# )
# task1.task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/PFI0")

# task2 = AITask("pressure", sample_rate=25600)
# task2.add_channel("p1", device_ind=0, channel_ind=1, ...)
# task2.task.timing.cfg_samp_clk_timing(...)
# task2.task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/PFI0")

# 2. Configure MultiHandler
# adv = MultiHandler()
# adv.configure(input_tasks=[task1, task2])
# adv.connect()

# 3. Acquire (blocks until trigger fires)
# data = adv.acquire()

# 4. Process results
# for task_name, channels in data.items():
#     for channel_name, samples in channels.items():
#         print(f"{task_name}/{channel_name}: {samples.shape}")

# 5. Clean up
# adv.disconnect()

print("See above for complete workflow.")

## Complete Example: Software-Triggered Single Task

This example demonstrates software-triggered acquisition with pyTrigger.

In [ ]:
# Pseudo-code for software-triggered acquisition

# 1. Create and configure task with CONTINUOUS mode (no hardware trigger)
# task = AITask("vibration", sample_rate=25600)
# task.add_channel("accel_x", device_ind=0, channel_ind=0, ...)
# task.start(start_task=False)  # Configure timing, but don't start yet

# 2. Configure MultiHandler with single input task
# adv = MultiHandler()
# adv.configure(input_tasks=[task])
# adv.connect()

# 3. Set trigger parameters
# adv.set_trigger(
#     n_samples=25600,        # Samples after trigger
#     trigger_channel=0,      # Monitor first channel
#     trigger_level=0.5,      # Threshold in physical units
#     trigger_type='abs',     # Absolute level
#     presamples=0            # No pre-trigger samples
# )

# 4. Acquire (blocks until trigger condition met)
# data = adv.acquire()
# print(data['accel_x'].shape)  # (25600,)
# print(data['time'].shape)     # (25600,)

# 5. Clean up
# adv.disconnect()

print("See above for complete workflow.")

## Error Handling

Common errors and how to resolve them:

**TypeError: input_tasks must be a list**
- `configure()` expects lists, even for single tasks: `configure(input_tasks=[task])`

**ValueError: Software trigger can only be used with one input task**
- Software triggering is single-task only; use hardware triggers for multi-task

**RuntimeError: set_trigger() must be called before acquire()**
- In software trigger mode, call `set_trigger()` before `acquire()`

**Validation failure (configure returns False)**
- Check that all tasks have the same sample rate within their group
- Verify timing configuration is identical across tasks
- Ensure acquisition mode matches trigger type (FINITE→hardware, CONTINUOUS→software)

## Key Differences from DAQHandler

| Feature | DAQHandler | MultiHandler |
|---------|------------|-------------|
| Inheritance | Standalone | Standalone (not a subclass) |
| Task management | Single AITask or AOTask | Multiple nidaqmx tasks |
| Task input | Accepts config dict | Accepts nidaqmx.Task, AITask, AOTask, or str |
| Trigger support | Software (pyTrigger) | Hardware OR software |
| Multi-task sync | No | Yes (hardware trigger) |
| Validation | Basic | Comprehensive pipeline |
| Return format | Dict (channel→data) | Dict (task→{channel→data}) |

**When to use:**

- Use `DAQHandler` for single-task scenarios with simple configuration
- Use `MultiHandler` for multi-task synchronization or complex trigger setups

## Summary

`MultiHandler` is the tool for advanced acquisition:

- Accepts mixed task types (nidaqmx.Task, AITask, AOTask, str)
- Validates configuration comprehensively before acquisition
- Supports hardware-triggered multi-task (FINITE) or software-triggered single-task (CONTINUOUS)
- Thread-safe with RLock protection

**Next steps:**

- Review `AITask` and `AOTask` notebooks for task creation
- Review `DAQHandler` notebook for simpler single-task scenarios
- Consult nidaqmx documentation for hardware trigger configuration